In [1]:
# Imports

########################################################################
# Python Standard Libraries
import os
import multiprocessing
from timeit import default_timer as timer
import random
import math

########################################################################
# Numpy Library
import numpy as np # linear algebra

########################################################################
# Pandas Library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

########################################################################
# MATPLOT Library
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator
%matplotlib inline

########################################################################
# SKLearn Library
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report, confusion_matrix, average_precision_score, roc_curve, auc, multilabel_confusion_matrix

########################################################################
# SCIPY Library
from scipy.stats import gaussian_kde
import scipy.stats as st


########################################################################
# Keras Library
from keras.models import Sequential
from keras.layers import Dense

########################################################################
# Init random seed
#seed = 13
#np.random.seed(seed)

In [2]:
# Utility functions
########################################################################
# Print system information
def print_system_info():
    mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')  # e.g. 4015976448
    mem_gib = mem_bytes/(1024.**3)  # e.g. 3.74
    print("{:<23}{:f} GB".format('RAM:', mem_gib))
    print("{:<23}{:d}".format('CORES:', multiprocessing.cpu_count()))
    !lscpu

########################################################################
# Walk through input files
def print_input_files():
    # Input data files are available in the "../input/" directory.
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

########################################################################
# Dump text files
def dump_text_file(fname):
    with open(fname, 'r') as f:
        print(f.read())

########################################################################
# Dump CSV files
def dump_csv_file(fname, count=5):
    # count: 0 - column names only, -1 - all rows, default = 5 rows max
    df = pd.read_csv(fname)
    if count < 0:
        count = df.shape[0]
    return df.head(count)

########################################################################
# Dataset related functions
ds_nbaiot = '/kaggle/input/nbaiot-dataset'
dn_nbaiot = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']

def fname(ds, f):
    if '.csv' not in f:
        f = f'{f}.csv'
    return os.path.join(ds, f)

def fname_nbaiot(f):
    return fname(ds_nbaiot, f)

def get_nbaiot_device_files():
    nbaiot_all_files = dump_csv_file(fname_nbaiot('data_summary'), -1)
    nbaiot_all_files = nbaiot_all_files.iloc[:,0:1].values
    device_id = 1
    indices = []
    for j in range(len(nbaiot_all_files)):
        if str(device_id) not in str(nbaiot_all_files[j]):
            indices.append(j)
            device_id += 1
    nbaiot_device_files = np.split(nbaiot_all_files, indices)
    return nbaiot_device_files

def get_nbaiot_device_data(device_id, count_norm=-1, count_anom=-1):
    if device_id < 1 or device_id > 9:
        assert False, "Please provide a valid device ID 1-9, both inclusive"
    if count_anom == -1:
        count_anom = count_norm
    device_index = device_id -1
    device_files = get_nbaiot_device_files()
    device_file = device_files[device_index]
    df = pd.DataFrame()
    y = []
    for i in range(len(device_file)):
        fname = str(device_file[i][0])
        df_c = pd.read_csv(fname_nbaiot(fname))
        count = count_anom
        if 'benign' in fname:
            count = count_norm
        count = min(count, df_c.shape[0]) if count >=0 else df_c.shape[0]
        rows = count
        print("processing", fname, "rows =", rows)
        y_np = np.ones(rows) if 'benign' in fname else np.zeros(rows)
        y.extend(y_np.tolist())
        df = pd.concat([df.iloc[:,:].reset_index(drop=True),
                      df_c.iloc[:rows,:].reset_index(drop=True)], axis=0)
    X = df.iloc[:,:].values
    y = np.array(y)
    Xdf = df
    return (X, y, Xdf)

def get_nbaiot_devices_data():
    devices_data = []
    for i in range(9):
        device_id = i + 1
        (X, y) = get_nbaiot_device_data(device_id)
        devices_data.append((X, y))
    return devices_data
#print_input_files()
print_system_info()

RAM:                   18.571461 GB
CORES:                 4
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx

In [3]:
def remove_correlated_features(df, threshold):
    df = df.copy()
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than a threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    # Drop features 
    df.drop(to_drop, axis=1, inplace=True)
    return df.iloc[:,:].values

def mark_important_features(vector, pc_keep): # pc_keep is the percentage (0-100) of labels to keep
    th = np.percentile(vector,(100-pc_keep)) # threshold, calculate percentile (100 - percentage) from percentage
    important_bool = (vector >= th)
    important_int = important_bool.astype(int)
    return important_int

def select_features(X, X_norm, X_anom, threshold):
    rows_n = X_norm.shape[0]
    rows_a = X_anom.shape[0]
    if rows_n == 0 or rows_a == 0:
        return X

    y_norm = np.ones(rows_n)
    y_anom = -1 * np.ones(rows_a)

    reg_n = LinearRegression(fit_intercept=False)
    reg_n.fit(X_norm, y_norm)
    coef_n = abs(reg_n.coef_)
    n = mark_important_features(coef_n, threshold)

    reg_a = LinearRegression(fit_intercept=False)
    reg_a.fit(X_anom, y_anom)
    coef_a = abs(reg_a.coef_)
    a = mark_important_features(coef_a, threshold)
   
    mask = np.bitwise_or(n,a)
    mask = mask == 1 # convert to Boolean
    X_sel = X[:, mask]
    return X_sel

In [4]:
def create_binary_classifier(hidden_layers, input_dim):
    layers = []
    for hl in hidden_layers:
        if hl > 0:
            layers.append(hl)

    layer_count = len(layers)
    assert layer_count >= 1, 'at least 1 non-zero hidden layer is needed'
    model = Sequential()
    model.add(Dense(layers[0],input_dim=input_dim,activation='relu'))
    for i in range(1, layer_count):
        model.add(Dense(layers[i],activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    #model.summary()
    model.compile(loss = 'binary_crossentropy',
             optimizer ='adam',metrics=['accuracy'])
    return model

In [5]:
def compute_time_complexity_single_pass(neurons_input, structure, neurons_output):
    count_hidden_layers = len(structure)
    neurons = [neurons_input, *structure, neurons_output]
    complexity = 0
    for i in range(count_hidden_layers + 1):
        complexity += neurons[i] * neurons[i+1]
    return complexity

In [6]:
def keep_mask(vector, pc_keep): # pc_keep is the percentage (0-100) of items to keep
    th = np.percentile(vector,(100-pc_keep)) # threshold, calculate percentile (100 - percentage) from percentage
    mask_bool = (vector >= th)
    mask_int = mask_bool.astype(int)
    return (mask_bool, mask_int)

In [7]:
'''
    a = mark_important_features(coef_a, threshold)
   
    mask = np.bitwise_or(n,a)
    mask = mask == 1 # convert to Boolean
    X_sel = X[:, mask]
    return X_sel
'''

'\n    a = mark_important_features(coef_a, threshold)\n   \n    mask = np.bitwise_or(n,a)\n    mask = mask == 1 # convert to Boolean\n    X_sel = X[:, mask]\n    return X_sel\n'

In [8]:
def analyze_ranks(model, threshold):
    w_d = 0
    b_d = 0
    layer_count = len(model.layers)
    layer_ranks = []
    new_structure = []
    for i in range(layer_count-1):
        (w_c, b_c) = model.layers[i+1].get_weights() # current weigths and biases
        rows = w_c.shape[0]
        cols = w_c.shape[1]
        w_c_abs = np.absolute(w_c)
        rank_neurons = np.sum(w_c_abs, axis=1) / cols
        rank_layer = np.sum(rank_neurons) / rows
        layer_ranks.append(rank_layer)
        (mask_bool, mask_int) = keep_mask(rank_neurons, threshold)
        new_neuron_count = np.sum(mask_int)
        new_structure.append(new_neuron_count)
        #print(f'{i+1}: {rank_neurons.shape} {rank_layer} {mask_int} {new_neuron_count}')
    layer_ranks = np.array(layer_ranks)
    new_structure = np.array(new_structure)
    (mask_bool, mask_int) = keep_mask(layer_ranks, threshold)
    new_structure = new_structure[mask_bool]
    #new_structure_masked = new_structure
    #print(f'layer mask: {mask_int} {new_structure}')
    return new_structure

In [9]:
model = None
def compute_report(title, model, X, y):
    y_pred = model.predict(X)
    y_pred = (y_pred > 0.25)
    #print(y_pred)
    #y_pred[y_pred <= 0] = -1 # convert negative values as 0 for anomaly
    #y_pred[y_pred > 0] = 1 # convert positive values as 1 for normal
    acc = accuracy_score(y, y_pred)
    tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
    cr = classification_report(y, y_pred)
    print("title,acc,tn,fp,fn,tp")
    print(f'{title}-cm,{acc:.2f},{tn},{fp},{fn},{tp}')
    #print(f'{cr}')
    results = model.evaluate(X, y, verbose=0)
    print(f'{title}-eval,{results}')
    return results[1] # accuracy

def rank_computation(title, X, y, structure, fc):
    global model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    print (f"========{title}========")
    feature_count = X.shape[1]
    print(f'Features={feature_count}')
    print(f'NN Structure: layers={len(structure)}, neurons: {structure}')
    complexity = compute_time_complexity_single_pass(fc, structure, 1)
    print(f'complexity: {complexity}')
    model = create_binary_classifier(structure, feature_count)
    model.fit(X_train,y_train,epochs=150,batch_size=10,verbose=0)
    #analyze_ranks(model, 95)
    acc_t = compute_report('training', model, X_train, y_train)
    acc_v = compute_report('validation', model, X_test, y_test)
    return (complexity, acc_t, acc_v)

In [10]:
output_title = ''
output_cplx_all_devices = []  # time complexity
output_acc_t_all_devices = [] # training accuracy
output_acc_v_all_devices = [] # validation accuracy
output_neurons_count_all_devices = [] # neurons count

In [11]:
debug_flag = True

In [12]:
if debug_flag:
    count_norm = 1000
    count_anom = 100
    #device_indices = [0]
    device_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8]
else:
    count_norm = -1
    count_anom = -1
    device_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8]

structures = {
    '0': [97, 48, 73, 58, 64, 53, 83],
    '1': [88, 52, 66, 58, 75],
    '2': [57, 31, 43, 38, 34, 49],
    '3': [101, 46, 67, 55, 60, 50, 75],
    '4': [87, 43, 58, 52, 47, 75],
    '5': [88, 44, 58, 52, 48, 75],
    '6': [59, 32, 44, 39, 35, 50],
    '7': [76, 48, 59, 53, 67],
    '8': [88, 48, 59, 53, 76]
}

fcs = [47, 47, 59, 51, 40, 42, 56, 56, 56]
for i in device_indices:
    global output_title
    device_index = i
    device_id = device_index + 1
    device_name = dn_nbaiot[device_index]
    (X, y, Xdf) = get_nbaiot_device_data(device_id, count_norm, count_anom)
    X = remove_correlated_features(Xdf, 0.98)
    X_std = StandardScaler().fit_transform(X)
    indices_norm = np.where(y >= 0.5)
    indices_anom = np.where(y <= 0.5)
    X_norm_all = X_std[indices_norm]
    X_anom_all = X_std[indices_anom]
    X_std = select_features(X_std, X_norm_all, X_anom_all,75)
    fc = fcs[i]
    print(f'############################[{i+1} - {device_name}]##########################')
    (complexity, acc_t, acc_v) = rank_computation(device_name, X_std, y, structures.get(f'{i}'), fc)
    thresholds = [100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85]
    output_title = 'device_name'
    output_cplx = f'{device_name}'
    output_acc_t = f'{device_name}'
    output_acc_v = f'{device_name}'
    output_neurons_count = f'{device_name}'
    main_model = [model]
    main_model = main_model[0]
    for th_i in range(30):
        threshold = 100 - th_i
        output_title = f'{output_title},{threshold}'
        new_structure = analyze_ranks(main_model, threshold)
        new_complexity = compute_time_complexity_single_pass(fc, new_structure, 1)
        neurons_count = sum(new_structure)
        print("........................................")
        print(f'Th={threshold}, Revised NN Structure: layers={len(new_structure)}, neurons: {new_structure}, complexity: {new_complexity}, neuron_count: {neurons_count}')
        #(complexity, acc_t, acc_v) = rank_computation(f'{device_name}-{threshold}', X_std, y, new_structure, fc)
        #output_cplx = f'{output_cplx},{complexity}'
        #output_acc_t = f'{output_acc_t},{acc_t}'
        #output_acc_v = f'{output_acc_v},{acc_v}'
        output_neurons_count = f'{output_neurons_count},{neurons_count}'
    #output_cplx_all_devices.append(output_cplx)
    #output_acc_t_all_devices.append(output_acc_t)
    #output_acc_v_all_devices.append(output_acc_v)
    output_neurons_count_all_devices.append(output_neurons_count)

processing 1.benign.csv rows = 1000
processing 1.gafgyt.combo.csv rows = 100
processing 1.gafgyt.junk.csv rows = 100
processing 1.gafgyt.scan.csv rows = 100
processing 1.gafgyt.tcp.csv rows = 100
processing 1.gafgyt.udp.csv rows = 100
processing 1.mirai.ack.csv rows = 100
processing 1.mirai.scan.csv rows = 100
processing 1.mirai.syn.csv rows = 100
processing 1.mirai.udp.csv rows = 100
processing 1.mirai.udpplain.csv rows = 100
############################[1 - Danmini_Doorbell]##########################
========Danmini_Doorbell========
Features=43
NN Structure: layers=7, neurons: [97, 48, 73, 58, 64, 53, 83]
complexity: 28539
title,acc,tn,fp,fn,tp
training-cm,0.99,660,1,6,673
training-eval,[0.01908925548195839, 0.9940298795700073]
title,acc,tn,fp,fn,tp
validation-cm,0.99,338,1,3,318
validation-eval,[0.04207624867558479, 0.9924242496490479]
........................................
Th=100, Revised NN Structure: layers=7, neurons: [97 48 73 58 64 53 83], complexity: 28539, neuron_count: 47

In [13]:
def print_summary(title, table_header, record):
    print(f"-------------------------{title}-------------------------")
    print(table_header)
    for item in record:
        print(item)

In [14]:
print(f"****************************[Summary]****************************")
#print_summary('time complexities', output_title, output_cplx_all_devices)
#print_summary('training accuracy', output_title, output_acc_t_all_devices)
#print_summary('validation accuracy', output_title, output_acc_v_all_devices)
print_summary('neurons count', output_title, output_neurons_count_all_devices)

****************************[Summary]****************************
-------------------------neurons count-------------------------
device_name,100,99,98,97,96,95,94,93,92,91,90,89,88,87,86,85,84,83,82,81,80,79,78,77,76,75,74,73,72,71
Danmini_Doorbell,476,387,382,379,375,371,367,363,359,355,351,348,344,339,336,332,327,281,277,275,270,267,264,260,256,255,251,247,244,241
Ecobee_Thermostat,339,260,256,255,251,249,246,244,241,239,236,233,231,228,225,223,220,218,215,213,210,208,204,202,200,197,152,150,147,146
Ennio_Doorbell,252,213,212,209,207,205,203,201,199,196,195,191,190,188,186,183,183,178,177,175,173,133,130,130,127,126,125,122,121,119
Philips_B120N10_Baby_Monitor,454,373,369,366,362,358,355,350,347,343,340,336,332,328,324,321,317,271,269,265,263,259,257,252,250,246,242,240,236,232
Provision_PT_737E_Security_Camera,362,310,306,303,300,297,293,291,288,285,281,278,275,272,269,266,263,260,256,254,250,189,186,184,181,179,177,174,172,170
Provision_PT_838_Security_Camera,365,285,282,279,276,2